In [16]:
import OT2_functions as OT
import cv2
import requests
import json
import time
import os
import subprocess
import numpy as np
import detection_functions as DF

In [ ]:
#Load labware

TIP_RACK = "opentrons_96_tiprack_300ul"
RESERVOIR = "nest_96_wellplate_200ul_flat"
PIPETTE = "p300_multi"
FAKE_PLATE = "agilent_1_reservoir_290ml"

run_id, commands_url = OT.create_run()

#pipette is 0, other labware is 1. Location is the location on the deck.
pipette_id = OT.load_equipment(0,PIPETTE)
reservoir_id = OT.load_equipment(1,RESERVOIR,'4')
tiprack_id = OT.load_equipment(1,TIP_RACK,'11')
fake_plate_id = OT.load_equipment(1,FAKE_PLATE,'6')

In [18]:
#camera calibration

#trun on the light
OT.light(True)

#take tips
#OT.pick_up(tiprack_id)
#OT.move(fake_plate_id, 50)
#OT.drop_tips(tiprack_id)
#OT.drop_tips(None)


cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
#OT.home()
OT.light(False)

Request status:
<Response [200]>
{"on":true}
Request status:
<Response [200]>
{"on":true}
Request status:
<Response [200]>
{"on":false}
Request status:
<Response [200]>
{"on":false}


In [8]:
#start

#image save
source = '/home/pi/Desktop/object_detection/image'# where the pic save
image_name = 'test-name'

#detect file
file_path = '/home/pi/Desktop/object_detection/detect_label_single.py'

In [12]:
# Take picture of the liquid level

OT.light(True)
#OT.pick_up(tiprack_id)
#OT.aspirate(reservoir_id)

OT.move(fake_plate_id, 50)

time.sleep(2)
DF.take_photo('liquid_level.jpg')
time.sleep(2)

image_path_ll = os.path.join(source, 'liquid_level.jpg') #path of the liquid level image
print(image_path_ll)
arguments_txt = ["--source", image_path_ll]
completed_process = subprocess.run(["python", file_path] + arguments_txt, capture_output=True, text=True)

output = completed_process.stdout
tips,liquids = DF.detect(output)
threshold = DF.save_threshold(tips,liquids)
#OT.dispense(reservoir_id)
#OT.drop_tips(tiprack_id)
OT.home()
OT.light(False)

Request status:
<Response [200]>
{"on":true}
Request status:
<Response [200]>
{"on":true}
Command:
{"data": {"commandType": "moveToWell", "params": {"labwareId": "e244fb17-e9ab-4c61-ae52-413986518bac", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 50, "y": 0, "z": 80}}, "pipetteId": "224465df-0ebc-47e0-9bbc-d7c49a19da1f"}, "intent": "setup"}}

Response:
<Response [201]>
{"data": {"id": "e6a5c9cf-d0d9-4f20-bff4-f9fb0fd799d1", "createdAt": "2023-07-13T14:05:35.765728+00:00", "commandType": "moveToWell", "key": "e6a5c9cf-d0d9-4f20-bff4-f9fb0fd799d1", "status": "queued", "params": {"forceDirect": false, "labwareId": "e244fb17-e9ab-4c61-ae52-413986518bac", "wellName": "A1", "wellLocation": {"origin": "top", "offset": {"x": 50.0, "y": 0.0, "z": 80.0}}, "pipetteId": "224465df-0ebc-47e0-9bbc-d7c49a19da1f"}, "intent": "setup"}}

finish
/home/pi/Desktop/object_detection/image/liquid_level.jpg
{}

Command:
{"target": "robot"}
Response:
<Response [200]>
{"message":"Homing rob

In [ ]:
## LOOP START
for i in range(20):
    source_name = f'{image_name}_{i+1}'
    source_path = f'{source}{source_name}'
    arguments = ["--source", source_path,'--save-txt']
    
    #take tips
    OT.pick_up(tiprack_id)
    #go to camera
    OT.aspirate(reservoir_id)
    OT.move(fake_plate_id, 50)
    time.sleep(2)
    DF.take_photo('source_name')
    time.sleep(2)#Overwrite the previous image file
    
    completed_process = subprocess.run(["python", file_path] + arguments, capture_output=True, text=True)
    output = completed_process.stdout
    tips,liquids = DF.detect(output)
    DF.match_boxes(tips,liquids,threshold)
    
    
    OT.dispense(reservoir_id)
    OT.drop_tips(tiprack_id)
    
    
## Turn on lights while running run
OT.home()
OT.light(False)

print('Your protocol is finished well done!')